In [1]:
import pandas as pd
import numpy as np

# pandas display preferences
pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 3)
#pd.option_context('display.max_rows', None)

#### Get the data:

In [2]:
# get the data

intakes = pd.read_csv('aac_intakes_20220304.csv')
outcomes = pd.read_csv('aac_outcomes_20220304.csv')

#### Rename Columns:

In [3]:
# rename columns

for col in intakes.columns:
    intakes = intakes.rename(columns={col: f'{col.lower().replace(" ", "_")}'})
    
for col in outcomes.columns:
    outcomes = outcomes.rename(columns={col: f'{col.lower().replace(" ", "_")}'})
    
cols = ['name', 'datetime', 'monthyear', 'animal_type', 'breed', 'color']
for col in cols:
    intakes = intakes.rename(columns={col: col+'_intake'})
    outcomes = outcomes.rename(columns={col: col+'_outcome'})


In [4]:
intakes.head(2)

,animal_id,name_intake,datetime_intake,monthyear_intake,found_location,intake_type,intake_condition,animal_type_intake,sex_upon_intake,age_upon_intake,breed_intake,color_intake
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver


In [5]:
outcomes.head(2)

,animal_id,name_outcome,datetime_outcome,monthyear_outcome,date_of_birth,outcome_type,outcome_subtype,animal_type_outcome,sex_upon_outcome,age_upon_outcome,breed_outcome,color_outcome
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown


#### Consider:  
Why are there more entries in each table than there are unique entries for animal_id? Perhaps that represents when the same animal has been seen more than once. 

In [6]:
len(outcomes)

137221

In [7]:
outcomes.animal_id.nunique()

122702

In [8]:
len(intakes)

136922

In [9]:
intakes.animal_id.nunique()

122390

#### Let's look at instances where animal_id exists more than once:

It does appear that duplicate animal ids represent times when an animal has been seen on more than one occasion. 

In [10]:
intakes_duplicated = intakes[intakes.animal_id.duplicated(keep=False)].sort_values('animal_id')
outcomes_duplicated = outcomes[outcomes.animal_id.duplicated(keep=False)].sort_values('animal_id')

In [11]:
intakes_duplicated.head()

,animal_id,name_intake,datetime_intake,monthyear_intake,found_location,intake_type,intake_condition,animal_type_intake,sex_upon_intake,age_upon_intake,breed_intake,color_intake
20227,A006100,Scamp,12/07/2017 02:07:00 PM,December 2017,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
91394,A006100,Scamp,03/07/2014 02:26:00 PM,March 2014,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
4342,A006100,Scamp,12/19/2014 10:21:00 AM,December 2014,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
99456,A245945,Boomer,07/03/2014 05:55:00 PM,July 2014,Garden And Mildred in Austin (TX),Stray,Normal,Dog,Neutered Male,14 years,Labrador Retriever Mix,Tan
122619,A245945,Boomer,05/20/2015 10:34:00 PM,May 2015,7403 Blessing Ave in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever Mix,Tan


In [12]:
outcomes_duplicated.head()

,animal_id,name_outcome,datetime_outcome,monthyear_outcome,date_of_birth,outcome_type,outcome_subtype,animal_type_outcome,sex_upon_outcome,age_upon_outcome,breed_outcome,color_outcome
115207,A006100,Scamp,03/08/2014 05:10:00 PM,Mar 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White
101468,A006100,Scamp,12/07/2017 12:00:00 AM,Dec 2017,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White
56676,A006100,Scamp,12/20/2014 04:35:00 PM,Dec 2014,07/09/2007,Return to Owner,NaN,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White
90674,A245945,Boomer,07/04/2014 03:26:00 PM,Jul 2014,05/23/2000,Return to Owner,NaN,Dog,Neutered Male,14 years,Labrador Retriever Mix,Tan
69515,A245945,Boomer,05/25/2015 11:49:00 AM,May 2015,05/23/2000,Transfer,Partner,Dog,Neutered Male,15 years,Labrador Retriever Mix,Tan


Due to difficulty in joining intakes with outcomes for these intances, we're going to drop these animals for now. Later evaluation would include these instances and add data about how many times the animal has been seen previously. 

#### drop the duplicate animals

In [13]:
intakes = intakes.loc[~ intakes.animal_id.isin(intakes_duplicated.animal_id)]
outcomes = outcomes.loc[~ outcomes.animal_id.isin(outcomes_duplicated.animal_id)]

#### Consider:
Why are there more outcomes than intakes?

In [14]:
len(intakes)

111273

In [15]:
len(outcomes)

111586

In [16]:
len(outcomes) - len(intakes)

313

Let's take a look at instances where an animal_id is represented in outcomes but not intakes

In [17]:
outcomes_without_intake = outcomes.loc[~ outcomes.animal_id.isin(intakes.animal_id)]
outcomes_without_intake

,animal_id,name_outcome,datetime_outcome,monthyear_outcome,date_of_birth,outcome_type,outcome_subtype,animal_type_outcome,sex_upon_outcome,age_upon_outcome,breed_outcome,color_outcome
80,A657197,Gecko,10/09/2013 01:15:00 PM,Oct 2013,05/12/2013,Adoption,NaN,Cat,Neutered Male,4 months,Domestic Shorthair Mix,Brown Tabby
111,A651995,*Sugar,07/02/2018 03:34:00 PM,Jul 2018,10/18/2011,Adoption,Foster,Dog,Spayed Female,6 years,Staffordshire Mix,Brown/White
157,A662741,*Todd,10/05/2013 12:42:00 PM,Oct 2013,08/11/2013,Transfer,Partner,Cat,Intact Male,1 month,Domestic Shorthair Mix,Black
915,A664032,Parker,10/01/2013 05:18:00 PM,Oct 2013,07/27/2013,Adoption,NaN,Dog,Spayed Female,2 months,German Shepherd Mix,Black/Tan
943,A662086,*Hanna,10/12/2013 04:01:00 PM,Oct 2013,08/29/2011,Adoption,NaN,Dog,Spayed Female,2 years,Labrador Retriever/Collie Smooth,Tan/White
...,...,...,...,...,...,...,...,...,...,...,...,...
137148,A841709,Brownie,09/03/2021 05:08:00 PM,Sep 2021,05/01/2021,Rto-Adopt,NaN,Dog,Neutered Male,4 months,Anatol Shepherd Mix,Brown/White
137162,A849583,Roxy,01/15/2022 05:20:00 PM,Jan 2022,12/18/2021,Adoption,NaN,Dog,Spayed Female,4 weeks,Labrador Retriever Mix,Brown Brindle/White
137198,A846552,Hamilton,11/20/2021 12:36:00 PM,Nov 2021,09/10/2021,Adoption,NaN,Dog,Neutered Male,2 months,Harrier Mix,White/Brown Brindle
137215,A842472,Beefy,09/18/2021 12:22:00 PM,Sep 2021,11/16/2020,Adoption,NaN,Dog,Spayed Female,10 months,Australian Cattle Dog/Pit Bull,Brown/White


no reason is immediately apparent. For now, we will drop these rows also. 


#### drop outcomes that don't have a corresponding intake

In [18]:
outcomes = outcomes.loc[~ outcomes.animal_id.isin(outcomes_without_intake.animal_id)]

In [19]:
len(intakes)

111273

In [20]:
len(outcomes)

110679

#### Consider:
Now we have more intakes than outcomes. Why?

In [21]:
intakes_without_outcomes = intakes.loc[~ intakes.animal_id.isin(outcomes.animal_id)]

Again, no reason is immediately apparent, so we will drop the rows for now. 

#### drop the intakes that do not have a corresponding outcome


In [22]:
intakes = intakes.loc[~ intakes.animal_id.isin(intakes_without_outcomes.animal_id)]

In [23]:
len(intakes)

110679

In [24]:
len(outcomes)

110679

 Now they match!

#### Join the dataframes

In [25]:
df = pd.merge(intakes, outcomes, on='animal_id')

#### Drop variables from original outcome table (except the target - 'outcome')
(by definition, not drivers of outcome, unless we start getting into duration of stay, etc. Which we wo't do at this point)

In [26]:
df.head()

,animal_id,name_intake,datetime_intake,monthyear_intake,found_location,intake_type,intake_condition,animal_type_intake,sex_upon_intake,age_upon_intake,breed_intake,color_intake,name_outcome,datetime_outcome,monthyear_outcome,date_of_birth,outcome_type,outcome_subtype,animal_type_outcome,sex_upon_outcome,age_upon_outcome,breed_outcome,color_outcome
0,A786884,*Brock,01/03/2019 04:19:00 PM,January 2019,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,*Brock,01/08/2019 03:11:00 PM,Jan 2019,01/03/2017,Transfer,Partner,Dog,Neutered Male,2 years,Beagle Mix,Tricolor
1,A706918,Belle,07/05/2015 12:59:00 PM,July 2015,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,Belle,07/05/2015 03:13:00 PM,Jul 2015,07/05/2007,Return to Owner,NaN,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver
2,A724273,Runster,04/14/2016 06:43:00 PM,April 2016,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,Runster,04/21/2016 05:17:00 PM,Apr 2016,04/17/2015,Return to Owner,NaN,Dog,Neutered Male,1 year,Basenji Mix,Sable/White
3,A665644,NaN,10/21/2013 07:59:00 AM,October 2013,Austin (TX),Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,NaN,10/21/2013 11:39:00 AM,Oct 2013,09/21/2013,Transfer,Partner,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico
4,A682524,Rio,06/29/2014 10:38:00 AM,June 2014,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,Rio,07/02/2014 02:16:00 PM,Jul 2014,06/29/2010,Return to Owner,NaN,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray


In [27]:
columns = ['datetime_outcome', 'monthyear_outcome', 'date_of_birth', 'outcome_subtype', 'animal_type_outcome', 'sex_upon_outcome', 'age_upon_outcome', 'breed_outcome', 'color_outcome', 'name_outcome']
df = df.drop(columns=columns)

#### Split the month_year column to extract the month (proxy for time of year)

In [28]:
df['month_intake'] = df.monthyear_intake.str.split().apply(lambda row: row[0])
df = df.drop(columns='monthyear_intake')

#### Split the sex upon intake column into fixed/not fixed and sex

In [29]:
df['fixed'] = df.sex_upon_intake.map({'Neutered Male': True,
                                      'Spayed Female': True,
                                      'Intact Male': False,
                                      'Intact Female': False,
                                      'Unknown': 'unknown'})

In [30]:
df['sex'] = df.sex_upon_intake.map({'Neutered Male': 'male',
                                      'Spayed Female': 'female',
                                      'Intact Male': 'male',
                                      'Intact Female': 'female',
                                      'Unknown': 'unknown'})

In [31]:
df = df.drop(columns='sex_upon_intake')

#### Rename Columns

In [32]:
df = df.rename(columns={'animal_type_intake': 'animal_type',
                        'breed_intake': 'breed', 
                        'color_intake': 'color',
                        'name_intake': 'name'})

In [33]:
def check_mixed(breed):
    if 'Mix' in breed or '/' in breed:
        return True
    else:
        return False
    
df['breed_mixed'] = df.breed.apply(lambda row: check_mixed(row))

In [34]:
df['breed'] = df.breed.str.replace(' Mix', '')

In [35]:
def breed_split_1(breed):
    if len(breed.split('/')) == 1:
        return breed
    else:
        return breed.split('/')[0]

def breed_split_2(breed):
    if len(breed.split('/')) > 1:
        return breed.split('/')[1]
    else:
        return np.nan
    
def breed_split_3(breed):
    if len(breed.split('/')) > 2:
        return breed.split('/')[2]
    else:
        return np.nan

df['breed_1'] = df.breed.apply(breed_split_1)
df['breed_2'] = df.breed.apply(breed_split_2)
df['breed_3'] = df.breed.apply(breed_split_3)
df = df.drop(columns='breed')

In [36]:
df['age_number'] = df.age_upon_intake.str.split().apply(lambda row: int(row[0]))
df['age_units'] = df.age_upon_intake.str.split().apply(lambda row: row[1])
df['age_multiplier'] = df.age_units.map({'day': 1, 
                                         'days': 1, 
                                         'week': 7, 
                                         'weeks': 7,
                                         'month': 30, 
                                         'months': 30, 
                                         'year': 365, 
                                         'years': 365})
df['age_intake'] = df.age_number * df.age_multiplier
df['age_intake'] = df.age_intake.apply(lambda row: pd.Timedelta(days=row))
df = df.drop(columns=['age_number', 'age_units', 'age_multiplier', 'age_upon_intake'])

In [37]:
def color_split_1(color):
    if len(color.split('/')) == 1:
        return color
    else:
        return color.split('/')[0]

def color_split_2(color):
    if len(color.split('/')) > 1:
        return color.split('/')[1]
    else:
        return np.nan

df['color_1'] = df.color.apply(color_split_1)
df['color_2'] = df.color.apply(color_split_2)
df = df.drop(columns='color')

In [38]:
df['datetime_intake'] = pd.to_datetime(df.datetime_intake)

In [39]:
import pandas as pd
import numpy as np

def aac_prep(intakes, outcomes):

    # rename columns

    for col in intakes.columns:
        intakes = intakes.rename(columns={col: f'{col.lower().replace(" ", "_")}'})

    for col in outcomes.columns:
        outcomes = outcomes.rename(columns={col: f'{col.lower().replace(" ", "_")}'})

    cols = ['name', 'datetime', 'monthyear', 'animal_type', 'breed', 'color']
    for col in cols:
        intakes = intakes.rename(columns={col: col+'_intake'})
        outcomes = outcomes.rename(columns={col: col+'_outcome'})
        
    
    # drop animals that have more than one entry (animals that were taken in on more than one occasion) 
    # (we should remove this step in later analysis)
    
    intakes_duplicated = intakes[intakes.animal_id.duplicated(keep=False)].sort_values('animal_id')
    outcomes_duplicated = outcomes[outcomes.animal_id.duplicated(keep=False)].sort_values('animal_id')
    
    intakes = intakes.loc[~ intakes.animal_id.isin(intakes_duplicated.animal_id)]
    outcomes = outcomes.loc[~ outcomes.animal_id.isin(outcomes_duplicated.animal_id)]
    
    
    # drop outcomes that don't have a corresponding intake, and vis-versa
    
    outcomes_without_intake = outcomes.loc[~ outcomes.animal_id.isin(intakes.animal_id)]
    outcomes = outcomes.loc[~ outcomes.animal_id.isin(outcomes_without_intake.animal_id)]
    
    intakes_without_outcomes = intakes.loc[~ intakes.animal_id.isin(outcomes.animal_id)]
    intakes = intakes.loc[~ intakes.animal_id.isin(intakes_without_outcomes.animal_id)]
    
    
    # join the dataframes
    
    df = pd.merge(intakes, outcomes, on='animal_id')
    
    
    # drop variables from the original outcomes table (since by definition, they're not drivers of outcome)
    
    columns = ['datetime_outcome', 'monthyear_outcome', 'date_of_birth', 'outcome_subtype', 'animal_type_outcome', 'sex_upon_outcome', 'age_upon_outcome', 'breed_outcome', 'color_outcome', 'name_outcome']
    df = df.drop(columns=columns)
    
    
    # split the month_year column to extract the month (proxy for time of year), then drop month_year column
    
    df['month_intake'] = df.monthyear_intake.str.split().apply(lambda row: row[0])
    df = df.drop(columns='monthyear_intake')
    
    
    # split the sex_upon_intake column into fixed = True/False and sex = male/female
    # then drop the sex_upon_intake column
    
    df['fixed'] = df.sex_upon_intake.map({'Neutered Male': True,
                                      'Spayed Female': True,
                                      'Intact Male': False,
                                      'Intact Female': False,
                                      'Unknown': 'unknown'})
    df['sex'] = df.sex_upon_intake.map({'Neutered Male': 'male',
                                      'Spayed Female': 'female',
                                      'Intact Male': 'male',
                                      'Intact Female': 'female',
                                      'Unknown': 'unknown'})
    df = df.drop(columns='sex_upon_intake')
    
    
    # rename columns
    
    df = df.rename(columns={'animal_type_intake': 'animal_type',
                        'breed_intake': 'breed', 
                        'color_intake': 'color',
                        'name_intake': 'name'})
    
    # determine if breed is 'mixed'
        # based on whether the breed description contains the word "Mix"
        # and based on whether there is more than one breed listed in the description (separated by "/")
    # create new column breed_mixed = True/False
    # then remove the word "Mix" from the breed description
    
    def check_mixed(breed):
        if 'Mix' in breed or '/' in breed:
            return True
        else:
            return False

    df['breed_mixed'] = df.breed.apply(lambda row: check_mixed(row))
    df['breed'] = df.breed.str.replace(' Mix', '')

    # split the breed description into multiple columns when there is more than one listed
    # then drop the original breed column
    
    def breed_split_1(breed):
        if len(breed.split('/')) == 1:
            return breed
        else:
            return breed.split('/')[0]

    def breed_split_2(breed):
        if len(breed.split('/')) > 1:
            return breed.split('/')[1]
        else:
            return np.nan

    def breed_split_3(breed):
        if len(breed.split('/')) > 2:
            return breed.split('/')[2]
        else:
            return np.nan

    df['breed_1'] = df.breed.apply(breed_split_1)
    df['breed_2'] = df.breed.apply(breed_split_2)
    df['breed_3'] = df.breed.apply(breed_split_3)
    df = df.drop(columns='breed')
    
    
    # split the color descriptino into multiple columns when there is more than one listed
    # then drop the original color column
    
    def color_split_1(color):
        if len(color.split('/')) == 1:
            return color
        else:
            return color.split('/')[0]

    def color_split_2(color):
        if len(color.split('/')) > 1:
            return color.split('/')[1]
        else:
            return np.nan

    df['color_1'] = df.color.apply(color_split_1)
    df['color_2'] = df.color.apply(color_split_2)
    df = df.drop(columns='color')
    
    
    # convert age column into pandas timedelta (number of days)
    
    df['age_number'] = df.age_upon_intake.str.split().apply(lambda row: int(row[0]))
    df['age_units'] = df.age_upon_intake.str.split().apply(lambda row: row[1])
    df['age_multiplier'] = df.age_units.map({'day': 1, 
                                             'days': 1, 
                                             'week': 7, 
                                             'weeks': 7,
                                             'month': 30, 
                                             'months': 30, 
                                             'year': 365, 
                                             'years': 365})
    df['age_intake'] = df.age_number * df.age_multiplier
    df['age_intake'] = df.age_intake.apply(lambda row: pd.Timedelta(days=row))
    df = df.drop(columns=['age_number', 'age_units', 'age_multiplier', 'age_upon_intake'])
    
    # convert the date & time of the intake into a pandas datetime type
    
    df['datetime_intake'] = pd.to_datetime(df.datetime_intake)
    
    return df

In [40]:
df.intake_type.value_counts()

Stray                 80041
Owner Surrender       18640
Public Assist          5872
Wildlife               5273
Abandoned               616
Euthanasia Request      237
Name: intake_type, dtype: int64

In [41]:
df.animal_type.value_counts()

Dog          55736
Cat          47073
Other         7194
Bird           652
Livestock       24
Name: animal_type, dtype: int64

In [42]:
df.intake_condition.value_counts()

Normal        93278
Injured        6895
Sick           5484
Nursing        3824
Aged            397
Other           216
Neonatal        199
Medical         142
Feral           115
Pregnant         90
Behavior         29
Space             4
Med Attn          3
Med Urgent        2
Panleuk           1
Name: intake_condition, dtype: int64

In [43]:
df.outcome_type.value_counts()

Adoption           48066
Transfer           36506
Return to Owner    14993
Euthanasia          8651
Died                1245
Disposal             617
Rto-Adopt            505
Missing               55
Relocate              24
Name: outcome_type, dtype: int64

In [44]:
df.head()

,animal_id,name,datetime_intake,found_location,intake_type,intake_condition,animal_type,outcome_type,month_intake,fixed,sex,breed_mixed,breed_1,breed_2,breed_3,age_intake,color_1,color_2
0,A786884,*Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Transfer,January,True,male,True,Beagle,NaN,NaN,730 days,Tricolor,NaN
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Return to Owner,July,True,female,False,English Springer Spaniel,NaN,NaN,2920 days,White,Liver
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Return to Owner,April,False,male,True,Basenji,NaN,NaN,330 days,Sable,White
3,A665644,NaN,2013-10-21 07:59:00,Austin (TX),Stray,Sick,Cat,Transfer,October,False,female,True,Domestic Shorthair,NaN,NaN,28 days,Calico,NaN
4,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Return to Owner,June,True,male,True,Doberman Pinsch,Australian Cattle Dog,NaN,1460 days,Tan,Gray


In [45]:
# get dogs
df = df[df.animal_type == 'Dog']

In [46]:
# drop columns not used for modeling at this time
df = df.drop(columns=['datetime_intake', 'found_location', 'name', 'animal_id', 'breed_2', 'breed_3', 'color_2', 'name'])

In [47]:
# drop rows in order to focus on most common outcome types
df = df[df.outcome_type.isin(['Adoption', 'Transfer', 'Return to Owner'])]

In [48]:
df.head()

,intake_type,intake_condition,animal_type,outcome_type,month_intake,fixed,sex,breed_mixed,breed_1,age_intake,color_1
0,Stray,Normal,Dog,Transfer,January,True,male,True,Beagle,730 days,Tricolor
1,Stray,Normal,Dog,Return to Owner,July,True,female,False,English Springer Spaniel,2920 days,White
2,Stray,Normal,Dog,Return to Owner,April,False,male,True,Basenji,330 days,Sable
4,Stray,Normal,Dog,Return to Owner,June,True,male,True,Doberman Pinsch,1460 days,Tan
5,Owner Surrender,Normal,Dog,Return to Owner,February,True,male,True,Labrador Retriever,730 days,Chocolate


In [49]:
# columns to hot code
categorical_columns = ['fixed', 'breed_mixed', 'intake_type', 'intake_condition', 'animal_type', 'month_intake', 'sex', 'breed_1', 'color_1']
# hot coding dummy variables
for col in categorical_columns:
    dummy_df = pd.get_dummies(df[col],
                              prefix=df[col].name,
                              drop_first=True,
                              dummy_na=False)
    df = pd.concat([df, dummy_df], axis=1)
    # drop original column
    df = df.drop(columns=col)

In [50]:
# turn age_intake timedelta into float
df['age_intake'] = df.age_intake / pd.Timedelta(days=1)

In [51]:
df.head()

,outcome_type,age_intake,fixed_True,fixed_unknown,breed_mixed_True,intake_type_Euthanasia Request,intake_type_Owner Surrender,intake_type_Public Assist,intake_type_Stray,intake_type_Wildlife,intake_condition_Behavior,intake_condition_Feral,intake_condition_Injured,intake_condition_Med Attn,intake_condition_Medical,intake_condition_Neonatal,intake_condition_Normal,intake_condition_Nursing,intake_condition_Other,intake_condition_Pregnant,intake_condition_Sick,month_intake_August,month_intake_December,month_intake_February,month_intake_January,...,color_1_Blue Tiger,color_1_Brown,color_1_Brown Brindle,color_1_Brown Merle,color_1_Brown Tiger,color_1_Buff,color_1_Chocolate,color_1_Cream,color_1_Fawn,color_1_Gold,color_1_Gray,color_1_Liver,color_1_Liver Tick,color_1_Orange,color_1_Red,color_1_Red Merle,color_1_Red Tick,color_1_Ruddy,color_1_Sable,color_1_Silver,color_1_Tan,color_1_Tricolor,color_1_White,color_1_Yellow,color_1_Yellow Brindle
0,Transfer,730.0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,Return to Owner,2920.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,Return to Owner,330.0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Return to Owner,1460.0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,Return to Owner,730.0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
def aac_get_dogs(df):
    df = df[df.animal_type == 'Dog']
    return df

In [53]:
def aac_prep_for_modeling(df):
    # drop columns not used for modeling at this time
    df = df.drop(columns=['datetime_intake', 'found_location', 'name', 'animal_id', 'breed_2', 'breed_3', 'color_2', 'name'])
    # drop rows in order to focus on most common outcome types
    df = df[df.outcome_type.isin(['Adoption', 'Transfer', 'Return to Owner'])]
    # columns to hot code
    categorical_columns = ['fixed', 'breed_mixed', 'intake_type', 'intake_condition', 'animal_type', 'month_intake', 'sex', 'breed_1', 'color_1']
    # hot coding dummy variables
    for col in categorical_columns:
        dummy_df = pd.get_dummies(df[col],
                                  prefix=df[col].name,
                                  drop_first=True,
                                  dummy_na=False)
        df = pd.concat([df, dummy_df], axis=1)
        # drop original column
        df = df.drop(columns=col)
    # turn age_intake timedelta into float
    df['age_intake'] = df.age_intake / pd.Timedelta(days=1)
    return df